In [ ]:
# ! pip install kafka-python
# ! pip install confluent_kafka
# ! pip install boto3

In [ ]:
from confluent_kafka import Producer
import json
import logging
from kafka import KafkaConsumer
from kafka.errors import NoBrokersAvailable



# p = Producer({'bootstrap.servers': 'localhost:19092'})

# try:
#     with open('./trace.json') as f:
#         try:
#             data = json.load(f)
#             if isinstance(data, list):
#                 for item in data:
#                     jd = json.dumps(item)
#                     print(jd)
#                     p.produce('observai_main_traces_1', jd)
#                     p.flush()
#             else:
#                 logging.error("Expected a JSON array in the file.")
#         except json.JSONDecodeError as exc:
#             logging.error(f"JSON decoding error in file: {exc}")
#         except Exception as exc:
#             logging.error(f"Unexpected error in file: {exc}")

# except FileNotFoundError as exc:
#     logging.error(f"File not found: {exc}")
# except Exception as exc:
#     logging.error(f"Unexpected error: {exc}")




# try:
#     with open('./trace.json') as f:
#         try:
#             data = json.load(f)
#             jd = json.dumps(data)
#             print(jd)
#             p.produce('observai_main_traces_1', jd)
#             p.flush()
#         except json.JSONDecodeError as exc:
#             logging.error(f"JSON decoding error in file: {exc}")
#         except Exception as exc:
#             logging.error(f"Unexpected error in file: {exc}")

# except FileNotFoundError as exc:
#     logging.error(f"File not found: {exc}")
# except Exception as exc:
#     logging.error(f"Unexpected error: {exc}")


In [23]:
import json
import logging
from kafka import KafkaConsumer
from kafka.errors import NoBrokersAvailable
# Configure logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)  # Corrected __name__

In [24]:
topic = 'powerlevel'
bootstrap_servers = ['telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443']
group_id = 'sustaingroup'
# SSL Configuration
ssl_certfile = './telemetry_zagaopenshift.crt'
ssl_cafile = './telemetry_zagaopenshift.pem'
# security_protocol = 'SSL'
  # Replace with your consumer group ID

In [25]:
consumer = None
try:
        # Create KafkaConsumer
        consumer = KafkaConsumer(
            topic,
            bootstrap_servers=bootstrap_servers,
            value_deserializer=lambda v: v,  # Keep value as bytes for JSON decoding
            group_id=group_id,
            auto_offset_reset='earliest',
            consumer_timeout_ms=1000,
            security_protocol='SSL',  # Add SSL configuration if needed
            ssl_cafile=ssl_cafile,
            ssl_certfile=ssl_certfile
        )
        logger.info("Successfully connected to Kafka broker")
except NoBrokersAvailable as e:
    logger.error(f"No brokers available. Ensure the Kafka broker is running and accessible. Error: {e}")

except Exception as e:
    logger.error(f"Failed to create Kafka consumer. Error: {e}")

DEBUG:kafka.metrics.metrics:Added sensor with name connections-closed
DEBUG:kafka.metrics.metrics:Added sensor with name connections-created
DEBUG:kafka.metrics.metrics:Added sensor with name select-time
DEBUG:kafka.metrics.metrics:Added sensor with name io-time
DEBUG:kafka.client:Initiating connection to node bootstrap-0 at telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent-received
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name request-latency
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.latency
DEBUG:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=telemetry-ka

In [22]:
# def consume_messages(consumer):
#     try:
#         logger.info("Starting message consumption")
#         for message in consumer:
#             json_message = json.loads(message.value.decode('utf-8'))
#             # print(json_message) # Decode and parse JSON
#             # yield json_message  # Yield each JSON message
#             print(logger.debug(message.value))
#     except KeyboardInterrupt:
#         logger.info("Stopping consumer...")
#     except Exception as e:
#         logger.error(f"An error occurred: {e}")
#     finally:
#         consumer.close()
#         logger.info("Consumer closed")
# consume_messages(consumer)

INFO:__main__:Starting message consumption
DEBUG:kafka.coordinator:Sending group coordinator request for group sustaingroup to broker bootstrap-0
DEBUG:kafka.protocol.parser:Sending request GroupCoordinatorRequest_v0(consumer_group='sustaingroup')
DEBUG:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443 <connected> [IPv4 ('192.168.31.160', 443)]> Request 3: GroupCoordinatorRequest_v0(consumer_group='sustaingroup')
DEBUG:kafka.client:Sending metadata request MetadataRequest_v1(topics=['powerlevel']) to node bootstrap-0
DEBUG:kafka.protocol.parser:Sending request MetadataRequest_v1(topics=['powerlevel'])
DEBUG:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443 <connected> [IPv4 ('192.168.31.160', 443)]> Request 4: MetadataRequest_v1(topics=['powerlevel'])
DEBUG:kafka.protocol.parser:Received

In [ ]:
def consume_messages(consumer):
    try:
        logger.info("Starting message consumption")
        for message in consumer:
            json_message = json.loads(message.value.decode('utf-8'))  # Decode and parse JSON
            yield json_message  # Yield each JSON message
    except KeyboardInterrupt:
        logger.info("Stopping consumer...")
    except Exception as e:
        logger.error(f"An error occurred: {e}")
    finally:
        consumer.close()
        logger.info("Consumer closed")

if __name__ == "__main__":
    # Check if the consumer is correctly created and not None
    if consumer is not None:
        try:
            # Consume messages
            for json_response in consume_messages(consumer):
                print(json.dumps(json_response, indent=2))  # Pretty-print JSON response
                print("success")
        except Exception as e:
            logger.error(f"Error consuming messages: {e}")
        finally:
            consumer.close()
            logger.info("Consumer closed")
    else:
        logger.error("Consumer initialization failed. Exiting...")